In [13]:
import pandas as pd
import geopandas as gpd
import plotly.express as px  # (version 4.7.0)
import plotly.graph_objects as go
import matplotlib.pyplot as plt

from jupyter_dash import JupyterDash  # (version 1.12.0) pip install dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

In [49]:
# plot not working
from shapely import wkt
aoi = pd.read_csv("f:/binder-dashboard/dashboard/type_fields.csv", sep=',',  encoding='utf8')
aoi['geometry'] = aoi['geometry'].apply(wkt.loads)
aoi = gpd.GeoDataFrame(aoi, crs='epsg:31370')


df_merged = aoi.to_crs(4326) # convert the coordinate reference system to lat/long
lga_json = df_merged.__geo_interface__ #covert to geoJSON

MAPBOX_ACCESSTOKEN = 'pk.eyJ1IjoiZmxvcmlzYWJyYW1zIiwiYSI6ImNrbDl3MWxicjEzeHQyb25wMnRhbHcyM24ifQ.Yt7LoCo4O2s8EFWm4gYoBg'


# Set the data for the map
data = go.Choroplethmapbox(geojson = lga_json)  
# Set the layout for the map
layout = go.Layout(
    mapbox1 = dict( 
        center = dict(lat=50.79, lon=5.30),
        accesstoken = MAPBOX_ACCESSTOKEN, 
        zoom = 4))

# Generate the map
fig=go.Figure(data=data, layout=layout)
fig.show()


In [47]:
lga_json

{'type': 'FeatureCollection',
 'features': [{'id': '0',
   'type': 'Feature',
   'properties': {'Unnamed: 0': 0,
    'crop_class': 'feed crop',
    'crop_type': 'Meerjarige luzerne',
    'deposition': 16425513.93681752,
    'id': 23458.0,
    'rank SAW': 8,
    'rank prom': 7,
    'rank_topsi': 5,
    'soil_class': 'Loam',
    'soil_type': 'Droge leem'},
   'geometry': {'type': 'Polygon',
    'coordinates': (((5.304224179692393, 50.79360469857442),
      (5.304189624043609, 50.7935453965445),
      (5.304031006410849, 50.79341522501673),
      (5.304007225494542, 50.7934277372024),
      (5.303899790767319, 50.793484262004135),
      (5.303704633557282, 50.793586940819004),
      (5.303260511282921, 50.793272498450335),
      (5.302909438968651, 50.79299081083228),
      (5.302892952231971, 50.79297758183813),
      (5.302045786581038, 50.79326645666816),
      (5.3020538332476495, 50.79327259765235),
      (5.303291746377125, 50.794201851534346),
      (5.303364894384162, 50.794157926

In [10]:
app = JupyterDash(__name__)

In [14]:
# App layout
app.layout = html.Div([

    html.H1("Web Application Dashboards with Dash", style={'text-align': 'center'}),

    dcc.Dropdown(id="Crop",
                 options=[
                     {"label": "feed crop", "value": "feed crop"},
                     {"label": "fruit", "value": "fruit"},
                     {"label": "non-leafy vegetable", "value": "non-leafy vegetable"},
                     {"label": "cereals", "value": "cereals"},
                     {"label": "root crop", "value": "root crop"},
                     {"label": "pasture", "value": "pasture"},
                     {"label": "tubers", "value": "tubers"},
                 ],
                 multi=False,
                 value="cereals",
                 style={'width': "40%"}
                 ),

    html.Div(id='output_container', children=[]),
    html.Br(),

    dcc.Graph(id='my_map', figure={})

])


# ------------------------------------------------------------------------------
# Connect the Plotly graphs with Dash Components
@app.callback(
    [Output(component_id='output_container', component_property='children'),
     Output(component_id='my_map', component_property='figure')],
    [Input(component_id='Crop', component_property='value')]
)
def update_graph(option_slctd):
    print(option_slctd)
    print(type(option_slctd))

    container = "The crop chosen by user was: {}".format(option_slctd)

    dff = aoi.copy()
    dff = dff[dff["crop_class"] == option_slctd]

    # Plotly Express
    fig = px.choropleth(
        data_frame=dff,
        locationmode='USA-states',
        locations='state_code',
        scope="usa",
        color='Pct of Colonies Impacted',
        hover_data=['State', 'Pct of Colonies Impacted'],
        color_continuous_scale=px.colors.sequential.YlOrRd,
        labels={'Pct of Colonies Impacted': '% of Bee Colonies'},
        template='plotly_dark'
    )

    # Plotly Graph Objects (GO)
    # fig = go.Figure(
    #     data=[go.Choropleth(
    #         locationmode='USA-states',
    #         locations=dff['state_code'],
    #         z=dff["Pct of Colonies Impacted"].astype(float),
    #         colorscale='Reds',
    #     )]
    # )
    #
    # fig.update_layout(
    #     title_text="Bees Affected by Mites in the USA",
    #     title_xanchor="center",
    #     title_font=dict(size=24),
    #     title_x=0.5,
    #     geo=dict(scope='usa'),
    # )

    return container, fig


In [15]:
app.run_server(mode='jupyterlab')